<a href="https://colab.research.google.com/github/Allone-chy/xdfUnet/blob/main/Basic_Unet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import os
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from skimage.io import imread, imshow

IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 3

TRAIN_PATH = r'C:\Users\Allone_chy\Desktop\Unet\raw'
MASK_PATH = r"C:\Users\Allone_chy\Desktop\Unet\test"

train_ids = next(os.walk(TRAIN_PATH))

mask_ids = next(os.walk(MASK_PATH))
# mask_ids[2]

# len(train_ids[2])

[]

In [ ]:
X_train = np.zeros((len(train_ids[2]),IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype = np.uint8)

for n, id_ in tqdm(enumerate(train_ids[2]),total = len(train_ids[2])):
    path = r'C:/Users/guangwei/Downloads/raw-20220908T042625Z-001/raw/'
    img = imread(path+id_)
    X_train[n] = img

# imshow(X_train[0])
# plt.show()

Y_train = np.zeros((len(train_ids[2]),IMG_HEIGHT, IMG_WIDTH, 1), dtype = bool)

for n, mask_id_ in tqdm(enumerate(mask_ids[2]),total = len(mask_ids[2])):
    path = r'C:/Users/guangwei/Downloads/raw-20220908T042625Z-001/masks/'
    mask_ = imread(path + mask_id_)
    mask_ = np.expand_dims(mask_, axis = -1)
    Y_train[n] = mask_
    #print(n, mask_id_)

# imshow(np.squeeze(Y_train[0]))
# plt.show()



In [ ]:
inputs = tf.keras.layers.Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
s = tf.keras.layers.Lambda(lambda x:x /255)(inputs)

c1 = tf.keras.layers.Conv2D(16, (3,3), activation = 'relu', kernel_initializer = 'he_normal', padding = 'same')(s)
c1 = tf.keras.layers.Dropout(0.1)(c1)
c1 = tf.keras.layers.Conv2D(16, (3,3), activation = 'relu', kernel_initializer = 'he_normal', padding = 'same')(c1)
p1 = tf.keras.layers.MaxPooling2D((2,2))(c1)

c2 = tf.keras.layers.Conv2D(32, (3,3), activation = 'relu', kernel_initializer = 'he_normal', padding = 'same')(p1)
c2 = tf.keras.layers.Dropout(0.1)(c2)
c2 = tf.keras.layers.Conv2D(32, (3,3), activation = 'relu', kernel_initializer = 'he_normal', padding = 'same')(c2)
p2 = tf.keras.layers.MaxPooling2D((2,2))(c2)

c3 = tf.keras.layers.Conv2D(64, (3,3), activation = 'relu', kernel_initializer = 'he_normal', padding = 'same')(p2)
c3 = tf.keras.layers.Dropout(0.1)(c3)
c3 = tf.keras.layers.Conv2D(64, (3,3), activation = 'relu', kernel_initializer = 'he_normal', padding = 'same')(c3)
p3 = tf.keras.layers.MaxPooling2D((2,2))(c3)

c4 = tf.keras.layers.Conv2D(128, (3,3), activation = 'relu', kernel_initializer = 'he_normal', padding = 'same')(p3)
c4 = tf.keras.layers.Dropout(0.1)(c4)
c4 = tf.keras.layers.Conv2D(128, (3,3), activation = 'relu', kernel_initializer = 'he_normal', padding = 'same')(c4)
p4 = tf.keras.layers.MaxPooling2D((2,2))(c4)

c5 = tf.keras.layers.Conv2D(256, (3,3), activation = 'relu', kernel_initializer = 'he_normal', padding = 'same')(p4)
c5 = tf.keras.layers.Dropout(0.1)(c5)
c5 = tf.keras.layers.Conv2D(256, (3,3), activation = 'relu', kernel_initializer = 'he_normal', padding = 'same')(c5)

u6 = tf.keras.layers.Conv2DTranspose(128, (2,2), strides = (2,2), padding = 'same')(c5)
u6 = tf.keras.layers.concatenate([u6,c4])
c6 = tf.keras.layers.Conv2D(128, (3,3), activation = 'relu', kernel_initializer = 'he_normal', padding = 'same')(u6)
c6 = tf.keras.layers.Dropout(0.1)(c6)
c6 = tf.keras.layers.Conv2D(128, (3,3), activation = 'relu', kernel_initializer = 'he_normal', padding = 'same')(c6)

u7 = tf.keras.layers.Conv2DTranspose(64, (2,2), strides = (2,2), padding = 'same')(c6)
u7 = tf.keras.layers.concatenate([u7,c3])
c7 = tf.keras.layers.Conv2D(64, (3,3), activation = 'relu', kernel_initializer = 'he_normal', padding = 'same')(u7)
c7 = tf.keras.layers.Dropout(0.1)(c7)
c7 = tf.keras.layers.Conv2D(64, (3,3), activation = 'relu', kernel_initializer = 'he_normal', padding = 'same')(c7)

u8 = tf.keras.layers.Conv2DTranspose(32, (2,2), strides = (2,2), padding = 'same')(c7)
u8 = tf.keras.layers.concatenate([u8,c2])
c8 = tf.keras.layers.Conv2D(32, (3,3), activation = 'relu', kernel_initializer = 'he_normal', padding = 'same')(u8)
c8 = tf.keras.layers.Dropout(0.1)(c8)
c8 = tf.keras.layers.Conv2D(32, (3,3), activation = 'relu', kernel_initializer = 'he_normal', padding = 'same')(c8)

u9 = tf.keras.layers.Conv2DTranspose(16, (2,2), strides = (2,2), padding = 'same')(c8)
u9 = tf.keras.layers.concatenate([u9,c1])
c9 = tf.keras.layers.Conv2D(16, (3,3), activation = 'relu', kernel_initializer = 'he_normal', padding = 'same')(u9)
c9 = tf.keras.layers.Dropout(0.1)(c9)
c9 = tf.keras.layers.Conv2D(16, (3,3), activation = 'relu', kernel_initializer = 'he_normal', padding = 'same')(c9)
outputs = tf.keras.layers.Conv2D(1,(1,1), activation = 'sigmoid')(c9)

model = tf.keras.Model(inputs =[inputs], outputs = [outputs])



In [ ]:
model.compile(optimizer ='adam', loss ='binary_crossentropy', metrics = ['accuracy'])
# model.summary()

checkpointer = tf.keras.callbacks.ModelCheckpoint('model_for_allen_mouse_brain_ISH.h5', verbose = 1, save_best_only = True)

callbacks = [
    tf.keras.callbacks.EarlyStopping(patience = 2, monitor = 'val_loss'),
    tf.keras.callbacks.TensorBoard(log_dir = 'logs')]

results = model.fit(X_train, Y_train, validation_split = 0.1, batch_size = 16, epochs = 25, callbacks = callbacks)

preds_train = model.predict(X_train[:40], verbose = 1)
#preds_val = model.predict(X_train[-1],verbose = 1)
#X_train[1].shape
#temp_img = X_train[1]
#imshow(temp_img)
#plt.show()
preds_val= model.predict(X_train[41:50], verbose = 1)

imshow((preds_train[1]>0.01).astype(np.uint8))
plt.show()

imshow(X_train[1])
plt.show()